# Example for:
Transfer Learning Empirical Experiment from first 5 classes of MNIST into Shoes ([5, 7, 9]) in FashionMNIST
- Almost all the details about optimizers, training, lr, ... are in the params dict!
- And all experiment design details too! reinit, freeze, ..

### Setup and Hyperparams

gpu_id is which GPU to use

In [ ]:
# Specify which gpu
import os
gpu_id = 1
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

In [1]:
from utils import *

In [2]:
# device = torch.device("cuda" if torch.cuda.is_available() else "mps")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cuts depends on model architecture:
- If the model has 5 conv layers, we have 6 cuts as below: cut=0 means do not freeze any conv layers, cut=5 means freeze all conv layers

In [3]:
# cuts=0 means: end-to-end model if we are reinitializing
cuts = [0,1,2,3,4,5]

model architecture
* we don't have support for multiple dense layers yet
* all the conv layers have to have the same number of channels
* even when save_best=False, the trainer function still returns the best epoch -just does not save it

In [7]:
# Changes Here for the experiments
params = {
      # MODEL ARCHITECTURE PARAMS
      'depth': 5,         # number of conv layers
      'num_channels': 30, # num channels for CNN
      # 'hidden_dim_lin': 128,   # for 2 dense layers at the end
      'activation_function': nn.ReLU,
      'kernel_size': 3,
      # TRAINING PARAMS
      'device': device,
      'lr_pretrain': 0.001,   
      'lr_fine_tune': 0.001,  # CHANGE: if no layer-wise lr
      # 'lr_fine_tune_reinit': 0.001,         # CHANGE: if no layer-wise lr
      # 'lr_fine_tune_no_reinit': 0.0001,     # CHANGE: if layer-wise lr
      'num_train': 40,
      'early_stop_patience': 6,
      'save_best': False,
      'save_checkpoints': False,
      'is_cnn': True,
      'is_debug': False,
      'classification_report_flag': False,
      'batch_size':4096,
      # DATASET PARAMS
      'pre_train_classes': list(range(5)), # CHANGE: First 5 classes of MNIST
      'fine_tune_classes': [5, 7, 9],       # CHANGE: Shoes in FashionMNIST
      'val_split': 0.1,
      'num_workers': 0,
      'generate_dataset_seed': 42,
      # EXPERIMENT SETTING PARAMS
      'use_pooling': False,   # CHANGE
      'freeze': True,         # CHANGE: freeze the conv layers before the cut
      'reinit': True,         # CHANGE: reinit the conv lyers only after the cut
      'truncate': False,   # CHANGE: True for reinitialize both dense layers, False for reinit only the last dense layer
      'truncate': False
      }

The dataset (from MNIST to FashionMNIST)

In [5]:
root_dir = './data'  # Specify your data directory here
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataloader_wrapped = TransferLearningWrapper(params, datasets.MNIST, datasets.FashionMNIST, root_dir, transform=transform)

## Pretraining

Defining the model: see how the final classifier layer has 5 nodes, because the pretraining dataset has 5 classes.
- That means the number of nodes in the classifier layer will change according to the fine-tune dataset classes when we cut the model

In [8]:
#Create DNN model
pretrained_model = CustomCNN(params, dataloader_wrapped.output_dim)
pretrained_model.to(device)

CustomCNN(
  (conv0): Conv2d(1, 30, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act0): ReLU()
  (conv1): Conv2d(30, 30, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (conv2): Conv2d(30, 30, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (conv3): Conv2d(30, 30, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (conv4): Conv2d(30, 30, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): ReLU()
  (fc): Linear(in_features=23520, out_features=5, bias=True)
)

Either pretrain a new model, or load a suitable one! (we usually load one for the serious experiments)

In [9]:
# Train and evaluate
trainer = Trainer(pretrained_model, dataloader_wrapped, params["lr_pretrain"], params)
train_acc, test_acc, effective_epochs, checkpoints = trainer.train()

print(f"Final Training Accuracy: {train_acc:.4f}")
print(f"Final Test Accuracy: {test_acc:.4f}")

Epoch: 0 	Training Accuracy: 86.66%
Validation Accuracy: 86.37%
Epoch: 1 	Training Accuracy: 93.60%
Validation Accuracy: 93.72%
Epoch: 2 	Training Accuracy: 95.62%
Validation Accuracy: 95.00%
Epoch: 3 	Training Accuracy: 96.19%
Validation Accuracy: 95.49%
Epoch: 4 	Training Accuracy: 96.82%
Validation Accuracy: 96.11%
Epoch: 5 	Training Accuracy: 97.01%
Validation Accuracy: 96.24%
Epoch: 6 	Training Accuracy: 97.66%
Validation Accuracy: 97.16%
Epoch: 7 	Training Accuracy: 98.08%
Validation Accuracy: 97.58%
Epoch: 8 	Training Accuracy: 98.51%
Validation Accuracy: 98.01%
Epoch: 9 	Training Accuracy: 98.86%
Validation Accuracy: 98.56%
Epoch: 10 	Training Accuracy: 98.93%
Validation Accuracy: 98.50%
Epoch: 11 	Training Accuracy: 99.26%
Validation Accuracy: 98.89%
Epoch: 12 	Training Accuracy: 99.30%
Validation Accuracy: 98.86%
Epoch: 13 	Training Accuracy: 99.42%
Validation Accuracy: 98.89%
Epoch: 14 	Training Accuracy: 99.46%
Validation Accuracy: 98.95%
Epoch: 15 	Training Accuracy: 99.48

Saving and loading a model: Adjust the folder path!

In [ ]:
pretrained_model.load_state_dict(torch.load('pretrained_models/pretrained_0.001/pretrained_model_89percent.pth'))
pretrained_model.to(device)

Eval to make sure loading worked

In [10]:
eval(pretrained_model, device, dataloader_wrapped.val_loader, debug=True, classification_report_flag=True, is_cnn=True)


Average loss: 0.0247, Accuracy: 3036.0/3059 (99%)

              precision    recall  f1-score   support

     Class 0       1.00      1.00      1.00       544
     Class 1       0.99      0.99      0.99       670
     Class 2       0.99      0.99      0.99       631
     Class 3       0.99      0.99      0.99       613
     Class 4       0.99      1.00      1.00       601

    accuracy                           0.99      3059
   macro avg       0.99      0.99      0.99      3059
weighted avg       0.99      0.99      0.99      3059



0.9924812030075187

In [ ]:
# save model for later use
foldername = "FashionMNIST/pretrained_models/pretrained_0.001"
os.mkdir(foldername)
torch.save(pretrained_model.state_dict(), os.path.join(foldername, 'pretrained_model.pth'))

params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
#save params as well
with open(os.path.join(foldername, 'params.json'), 'w') as fp:
    json.dump(params_tmp, fp)

## Fine-tuning Experiments

### Baselines (End-to-end models trained on subsets of fine-tuning dataset)
We also reuse the baselines a lot! so skip if we already have the jsons

In [11]:
baselines_results = []
# percentages = [0.001, 0.002, 0.005, 0.01, 0.05, 0.1, 0.3, 0.5, 0.8, 1]
percentages = [0.001, 0.002, 0.005]

In [ ]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:      
    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
    
    for repeat in range(repeats):
        # Print or log the sampled values for transparency
        print(f"\nSampled Percentage: {sampled_percentage}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

        # Reduce the dataset
        train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed = repeat)
        torch.manual_seed(repeat)
        #train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed = repeat)
        dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)

        # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
        params_tmp = copy.deepcopy(params)
        params_tmp["reinit"] = True
        model_new = cut_custom_cnn_model(pretrained_model, cut_point=0, params=params_tmp, output_dim=dataloader_wrapped.output_dim)
        model_new.to(device)

        # Train and evaluate
        trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
        train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
        print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

        # Store the results
        baselines_results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":-1, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc}) # -1 for the cut point means it's baseline

Save the results in json: adjust the path

In [ ]:
# save baseline results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
baselines_results = [params_tmp] + baselines_results

with open(f'results_jsons/baselines_freeze_{params["freeze"]}_pool_{params["use_pooling"]}_lr_{params["lr_fine_tune"]}_dummy_run.json', 'w') as f:
    json.dump(baselines_results, f)

Load the results: can load them in visualizations.ipynb or statistical_tests.ipynb for later use

In [ ]:
#load results
with open('results_jsons/baselines_freeze_True_pool_False_lr_0.001_dummy_run.json', 'r') as f:
    results_baseline = json.load(f)
results_baseline = results_baseline[1:]

# make sure these are only the baselines
results_baseline = [r for r in results_baseline if r["sampled_cut_point"] == -1]

### Fine-tuning

In [ ]:
results = []
# percentages = [0.001, 0.002, 0.005, 0.01, 0.05, 0.1, 0.3, 0.5, 0.8, 1]
percentages = [0.001, 0.002, 0.005]

In [ ]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:

    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
        
    for sampled_cut_point in cuts:

        for repeat in range(repeats):
            # Add the combination to the tested set
            # tested_combinations.add((sampled_percentage, sampled_cut_point))

            # Print or log the sampled values for transparency
            print(f"\nSampled Percentage: {sampled_percentage}, Sampled Cut Point: {sampled_cut_point}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

            # Reduce the dataset
            train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed=repeat)
            dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)
            torch.manual_seed(repeat) # because in the cut function we reinitialize some layers too (at least the dense layers)
            
            # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
            model_new = cut_custom_cnn_model(pretrained_model, cut_point=sampled_cut_point, params=params, output_dim=dataloader_wrapped.output_dim)
            model_new.to(device)
            
            # Train and evaluate
            trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
            train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
            print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

            # Store the results
            results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":sampled_cut_point, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc})

In [ ]:
# save fine-tuning results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + results

with open(f'results_jsons/results_freeze_{params["freeze"]}_reinit_{params["reinit"]}_pool_{params["use_pooling"]}_truncate_{params["truncate"]}_lr_{params["lr_fine_tune"]}_{percentages[0]}_to_{percentages[-1]}.json', 'w') as f:
    json.dump(results, f)
results = results[1:]

In [ ]:
#load results
with open('results_jsons/results_freeze_True_pool_False_lr_0.001_dummy_run.json', 'r') as f:
    results = json.load(f)
results = results[1:]

# make sure these are only the baselines
results = [r for r in results if r["sampled_cut_point"] == -1]